# Galleria rapida delle tracce

Notebook derivato da `View_tracking_tiles` per effettuare una revisione visiva del dataset di tracking.
Consente di sfogliare più clip contemporaneamente e di annotare velocemente quelle da conservare.

In [1]:
import os, re
import numpy as np
import pandas as pd

from arguments import prepare_finetuning_args
from dataset.data_manager import BuildTrackingDataset, DataManager
import torch
from torch.utils.data import DataLoader
from dataset.datasets import MedicanesTrackDataset
from dataset.build_dataset import get_cyclone_center_pixel
from PIL import Image, ImageDraw
from io import BytesIO

import ipywidgets as widgets
from IPython.display import display
from typing import Sequence, List, Optional, Any

from view_test_tiles import VideoGallerySelector


Impossibile caricare la libreria torch dynamo


In [2]:
args = prepare_finetuning_args()

# Percorsi
input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised/"
csv_out = "train_tracking.csv"          # CSV finale con pixel per tracking
manos_file = "medicane_data_input/medicanes_new_windows.csv"
df_tracks = pd.read_csv(manos_file, parse_dates=['time', 'start_time', 'end_time'])

bt = BuildTrackingDataset(type="supervised", args=args)
bt.prepare_data(df_tracks, input_dir, output_dir)
bt.create_tracking_csv(output_dir, csv_out)


sorted_metadata_files num :  11145
1)  ->
Con cicloni: 29
Senza cicloni: 247
276 video per il periodo (effettivo) da 2010-10-12 13:45:00 a 2010-10-13 13:00:00

2)  ->
Con cicloni: 56
Senza cicloni: 616
672 video per il periodo (effettivo) da 2011-11-05 23:45:00 a 2011-11-08 08:00:00

3)  ->
Con cicloni: 42
Senza cicloni: 426
468 video per il periodo (effettivo) da 2012-04-13 05:45:00 a 2012-04-14 21:00:00

4)  ->
Con cicloni: 39
Senza cicloni: 369
408 video per il periodo (effettivo) da 2013-11-18 22:45:00 a 2013-11-20 09:00:00

5)  ->
Con cicloni: 49
Senza cicloni: 383
432 video per il periodo (effettivo) da 2014-11-07 01:45:00 a 2014-11-08 14:00:00

6)  ->
Con cicloni: 73
Senza cicloni: 767
840 video per il periodo (effettivo) da 2014-11-30 07:45:00 a 2014-12-03 06:00:00

7)  ->
Con cicloni: 25
Senza cicloni: 239
264 video per il periodo (effettivo) da 2015-09-30 22:45:00 a 2015-10-01 21:00:00

8)  ->
Con cicloni: 31
Senza cicloni: 341
372 video per il periodo (effettivo) da 2016-02-

In [3]:
# Costruisce DataLoader per tracking a partire dal CSV generato
dm = DataManager(is_train=False, args=args, type_t='supervised', specify_data_path=csv_out)
track_loader = dm.get_tracking_dataloader(args)
track_ds = dm.dataset


Getting TRACKING dataset (pixel coords)...
[INFO][TrackingDataset] Dropped 15 rows with missing/non-finite coordinates from train_tracking.csv (kept 982/997).
DATASET length: 982
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7f8b746eb250>
Batch_size local: 1


## Selezione simultanea di più clip

La classe seguente crea una galleria di clip video con pulsanti "toggle" per selezionare le tracce da mantenere.
Gli identificativi selezionati (indici o path) possono poi essere salvati e riutilizzati per filtrare il DataFrame del CSV.

In [7]:
selezionati = []
esclusi = []

In [78]:
982/36

27.27777777777778

In [86]:
# Estrae alcune clip dal dataset per popolare la galleria
i = 27
max_samples = 36
sample_indices = list(range(max_samples*i, max_samples*(i+1)))

clips = []
identifiers = []

mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1, 1)
marker_radius = 5
marker_color = (255, 0, 0)

for ds_idx in sample_indices:
    video, coords, folder = track_ds[ds_idx]
    video = (video * std + mean).clamp(0, 1)

    px_tile, py_tile = map(float, coords[:2])

    frames = []
    for frame in video.permute(1, 2, 3, 0).cpu().numpy():
        img = (np.clip(frame, 0.0, 1.0) * 255).astype('uint8')
        pil = Image.fromarray(img)
        draw = ImageDraw.Draw(pil)
        draw.ellipse(
            (
                px_tile - marker_radius,
                py_tile - marker_radius,
                px_tile + marker_radius,
                py_tile + marker_radius
            ),
            fill=marker_color
        )
        frames.append(np.asarray(pil) / 255.0)
    clips.append(frames)

    # Usa il path della cartella come identificativo per poterlo incrociare con il CSV
    identifiers.append(folder)

selector = VideoGallerySelector(
    clips,
    identifiers=identifiers,
    columns=6,
    interval=200,
    preview_width=140,
    grid_gap='8px'
)
selector.display()


IndexError: single positional indexer is out-of-bounds

In [84]:
esc = selector.get_excluded_identifiers()
esclusi.extend(esc)

In [13]:
# Recupera gli identificativi (es. path cartella) delle clip marcate
sel = selector.get_selected_identifiers()
selezionati.extend(sel)

In [87]:
len(esclusi),# esclusi

(163,)

In [54]:
len(selezionati), selezionati

(0, [])

## Uso con il DataFrame del CSV

Gli identificativi ottenuti possono essere utilizzati per filtrare il CSV originale.
Ad esempio, se il path della cartella è presente in una colonna `folder`, si può eseguire:

```python

df_selezionati = df_tracking[df_tracking['folder'].isin(selezionati)]
```

In alternativa è possibile memorizzare gli indici relativi della galleria tramite
`selector.get_selected_indices()` e unirli con l'indice del DataFrame.

In [89]:
df_train = pd.read_csv(csv_out)
df_train

,path,start,end,x_pix,y_pix
0,../airmassRGB/supervised/12-10-2010_1500_213_0,1,16,87.0,155.0
1,../airmassRGB/supervised/12-10-2010_1600_213_0,1,16,84.0,156.0
2,../airmassRGB/supervised/12-10-2010_1700_213_0,1,16,80.0,157.0
3,../airmassRGB/supervised/12-10-2010_1800_213_0,1,16,78.0,158.0
4,../airmassRGB/supervised/12-10-2010_1900_213_0,1,16,75.0,160.0
...,...,...,...,...,...
992,../airmassRGB/supervised/09-09-2023_2200_639_196,1,16,178.0,148.0
993,../airmassRGB/supervised/09-09-2023_2300_639_196,1,16,186.0,148.0
994,../airmassRGB/supervised/10-09-2023_0000_639_196,1,16,186.0,148.0
995,../airmassRGB/supervised/10-09-2023_0100_639_196,1,16,193.0,148.0


In [91]:
df_selezionati = df_train[~df_train['path'].isin(esclusi)]
df_selezionati

,path,start,end,x_pix,y_pix
4,../airmassRGB/supervised/12-10-2010_1900_213_0,1,16,75.0,160.0
5,../airmassRGB/supervised/12-10-2010_2000_213_0,1,16,73.0,161.0
6,../airmassRGB/supervised/12-10-2010_2100_213_0,1,16,70.0,164.0
7,../airmassRGB/supervised/12-10-2010_2200_213_0,1,16,67.0,168.0
8,../airmassRGB/supervised/12-10-2010_2300_213_0,1,16,66.0,171.0
...,...,...,...,...,...
992,../airmassRGB/supervised/09-09-2023_2200_639_196,1,16,178.0,148.0
993,../airmassRGB/supervised/09-09-2023_2300_639_196,1,16,186.0,148.0
994,../airmassRGB/supervised/10-09-2023_0000_639_196,1,16,186.0,148.0
995,../airmassRGB/supervised/10-09-2023_0100_639_196,1,16,193.0,148.0


In [92]:
df_selezionati.to_csv("train_tracking_selezionati.csv", index=False)